In [1]:
import os
#if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#    raise 'stop'
if not os.path.isdir('d:/flagellar/'):
    deps_path = '/kaggle/usr/lib/flg-packages/'
    !pip install --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt
    !tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
    !pip install --no-index --find-links=./packages ultralytics
    !rm -rf ./packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo

fast_mode = False
if fls.is_submission:
    fast_mode = False

In [3]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:30]
    test_data = test_data[2:4]
len(train_data), len(test_data)

(444, 200)

In [6]:
[d.name for d in fls.load_all_extra_data()]

['aba2013-04-06-10',
 'aba2013-04-06-12',
 'aba2013-04-06-13',
 'aba2013-04-06-14',
 'aba2013-04-06-16',
 'aba2013-04-06-17',
 'aba2013-04-06-18',
 'aba2013-04-06-19',
 'aba2013-04-06-20',
 'aba2013-04-06-21',
 'aba2013-04-06-22',
 'aba2013-04-06-7',
 'aba2013-04-06-8',
 'aba2013-04-06-9',
 'aba2013-12-24-1',
 'aba2013-12-24-21',
 'aba2013-12-24-7',
 'aba2013-12-25-14',
 'aba2013-12-25-17',
 'aba2013-12-25-18',
 'aba2013-12-25-6',
 'aba2014-02-21-11',
 'aba2014-02-21-12',
 'aba2014-02-21-13',
 'aba2014-02-21-14',
 'aba2014-02-21-15',
 'aba2014-02-21-16',
 'aba2014-02-21-17',
 'aba2014-02-21-18',
 'aba2014-02-21-19',
 'aba2014-02-21-2',
 'aba2014-02-21-24',
 'aba2014-02-21-26',
 'aba2014-02-21-27',
 'aba2014-02-21-3',
 'aba2014-02-21-5',
 'aba2014-02-21-6',
 'aba2014-02-21-8',
 'aba2014-02-21-9',
 'aba2014-03-04-1',
 'aba2014-03-04-10',
 'aba2014-03-04-11',
 'aba2014-03-04-12',
 'aba2014-03-04-13',
 'aba2014-03-04-17',
 'aba2014-03-04-18',
 'aba2014-03-04-19',
 'aba2014-03-04-2',
 'aba2

In [ ]:
importlib.reload(flg_yolo)
model = flg_yolo.YOLOModel()
model.seed = 42
model.use_pretrained_weights = False
model.n_epochs = 30
model.run_in_parallel = True
if fast_mode: model.n_epochs = 1
model.train(train_data, test_data)
if not fls.env=='kaggle':
    fls.dill_save(fls.model_dir+ "yolo.pickle", model)

42
['tomo_507b7a', 'tomo_b33d4e', 'tomo_b7d014', 'tomo_b0ded6', 'tomo_37dd38']
['tomo_7550f4', 'tomo_0fe63f', 'tomo_08446f', 'tomo_e5ac94', 'tomo_6f0ee4']
Will process approximately 3123 slices for training


Processing training motors:   0%|          | 0/347 [00:00<?, ?it/s]

Will process approximately 900 slices for validation


Processing validation motors:   0%|          | 0/100 [00:00<?, ?it/s]


Processing Summary:
- Train set: 258 tomograms, 347 motors, 3118 slices
- Validation set: 100 tomograms, 100 motors, 900 slices
- Total: 358 tomograms, 447 motors, 4018 slices

Preprocessing Complete:
- Training data: 258 tomograms, 347 motors, 3118 slices
- Validation data: 100 tomograms, 100 motors, 900 slices
- Dataset directory: d:/flagellar/temp//yolo_dataset/
- YAML configuration: d:/flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Directory status:
- Train images exists: True
- Val images exists: True
- Train labels exists: True
- Val labels exists: True
Found original dataset.yaml at d:/flagellar/temp//yolo_dataset/dataset.yaml
Fixing YAML paths in d:/flagellar/temp//yolo_dataset/dataset.yaml
Created fixed YAML at d:/flagellar/temp/fixed_dataset.yaml with path: d:/flagellar/temp//yolo_dataset/
Using YAML file: d:/flagellar/temp/fixed_dataset.yaml
YAML contents:
names:
  0: motor
path: d:/flagellar/temp//yolo_dataset/
train: 

...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07

                   all        900        900      0.919      0.829      0.887      0.332
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 20, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



30 epochs completed in 0.435 hours.
Optimizer stripped from d:\flagellar\temp\yolo_weights\motor_detector\weights\last.pt, 52.0MB
Optimizer stripped from d:\flagellar\temp\yolo_weights\motor_detector\weights\best.pt, 52.0MB

Validating d:\flagellar\temp\yolo_weights\motor_detector\weights\best.pt...
Ultralytics 8.3.98  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLOv8m summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs



                Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:12

                   all        900        900      0.771      0.779      0.835      0.366
Speed: 0.2ms preprocess, 8.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to d:\flagellar\temp\yolo_weights\motor_detector

Training complete!


In [5]:
if not fls.is_submission:
    importlib.reload(flg_yolo)    
    inferred_test_data = model.infer(test_data[:20])
    print(fls.score_competition_metric(inferred_test_data, test_data[:20]))

0.9016393442622951


In [6]:
inferred_test_data2 = model.infer(fls.load_all_test_data())
fls.write_submission_file(inferred_test_data2)

       tomo_id  Motor axis 0  Motor axis 1  Motor axis 2
0  tomo_003acc            -1            -1            -1
1  tomo_00e047           166           546           603
2  tomo_01a877           148           639           282


In [7]:
inferred_test_data2

[DataKaggle(is_train=False, name='tomo_003acc', labels=Empty DataFrame
 Columns: []
 Index: [], labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='unloaded', data=None, data_shape=(500, 1912, 1847), voxel_spacing=nan, mean_per_slice=array([], dtype=float64), std_per_slice=array([], dtype=float64), percentiles_per_slice=array([], shape=(8, 0), dtype=float64), resize_factor=1.0, target_size=None),
 DataKaggle(is_train=False, name='tomo_00e047', labels=     z    y    x
 0  166  546  603, labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='unloaded', data=None, data_shape=(300, 959, 928), voxel_spacing=nan, mean_per_slice=array([], dtype=float64), std_per_slice=array([], dtype=float64), percentiles_per_slice=array([], shape=(8, 0), dtype=float64), resize_factor=1.0, target_size=None),
 DataKaggle(is_train=False, name='tomo_01a877', labels=     z    y    x
 0  148  639  282, labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='u